In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: GerryChain, 1.0.2-alpha"
#r "nuget: Plotly.NET, 2.0.0-preview.17"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.17"

Installed Packages GerryChain, 1.0.2-alpha Newtonsoft.Json, 13.0.1 Plotly.NET, 2.0.0-preview.17 Plotly.NET.Interactive, 2.0.0-preview.17

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
using System;
using System.Linq;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using Plotly.NET;
using Plotly.NET.Interactive;
using Plotly.NET.LayoutObjects;
using GerryChain;
using PCompress;

In [ ]:
var TallyBVAP = Scores.TallyFactory("BVAP", "BVAP");
var TallyVAP = Scores.TallyFactory("VAP", "VAP");
var TallyPOP = Scores.TallyFactory("TOTPOP", "TOTPOP");
var initPart = new Partition("../resources/al_vtds20_with_seeds.json", "CD_Seed", "TOTPOP", new string[] { "TOTPOP", "VAP", "BVAP" },
                             new Score[] { TallyBVAP, TallyPOP, TallyVAP }, regionAware: true, regionDivisionSpecs: new (string, double)[]{("COUNTY", 1)});
var chain = new Chain(initPart, 10000, 0.05, batchSize:4);

In [ ]:
var bs = chain.Select(p => ((DistrictWideScoreValue) p.Score("BVAP")).Value.Zip(((DistrictWideScoreValue)p.Score("VAP")).Value, (b, v) => b/v)).ToArray();

## Recording a chain

In [ ]:
var rec = new Recorder(chain, "test.chain");
rec.Record();

## Replaying the previous chain

In [ ]:
var g = GraphParsers.LoadGraphFromJson("../resources/al_vtds20_with_seeds.json", "TOTPOP", new string[] { "TOTPOP", "VAP", "BVAP" });

In [ ]:
var replayer = new Replayer(g.graph, new Score[] { TallyBVAP, TallyPOP, TallyVAP }, "test.chain");

In [ ]:
// var dist1BVAPs = new double [1000];
var numCutEdges = new int[10000];
int i = 0;

foreach (Partition p in replayer.Replay())
{
    numCutEdges[i] = p.CutEdges.Count();
    // dist1BVAPs[i] = ((DistrictWideScoreValue)p.Score("BVAP")).Value [0];
    i++;
}

Error: System.InvalidCastException: Unable to cast object of type 'Newtonsoft.Json.Linq.JValue' to type 'Newtonsoft.Json.Linq.JArray'.
   at PCompress.Replayer.Replay(Boolean flips)+MoveNext()
   at Submission#8.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
GenericChart.GenericChart chart = Chart2D.Chart.Histogram<int,int,int>(X: numCutEdges, Orientation: StyleParam.Orientation.Vertical, NBinsX: 25)
    .WithXAxisStyle(title: Title.init("# Cut Edges"), ShowGrid: false, ShowLine: true)
    .WithYAxisStyle(title: Title.init("Frequency"), ShowGrid: false, ShowLine: true);
chart

<!-- Plotly chart will be drawn inside this DIV -->